# Task 3: Air Pollution Forecasting — Project Overview

This project focuses on building and evaluating a recurrent neural network (RNN) model for analyzing time series data describing air pollution (PM2.5) in Beijing during 2013–2017.  
The goal is to forecast PM2.5 concentration based on meteorological data from the Wanshouxigong monitoring station.

Project steps include:

- Loading, cleaning, and exploring the CSV dataset
- Preparing data for time series analysis (sequential split, standardization, TimeseriesGenerator)
- Building and training an LSTM (Long Short-Term Memory) model
- Evaluating prediction quality using regression metrics (MSE, R²)
- Visualizing results and interpreting the model.


In [ ]:
## Import required libraries